In [1]:
include("graph.jl")
include("base_operators.jl")

include("Convolution.jl")
include("Dense.jl")
include("Flatten.jl")
include("MaxPool.jl")
include("ReLu.jl")

include("misc.jl")

onecold (generic function with 1 method)

In [2]:
# 
import Pkg
Pkg.add("BenchmarkTools")
Pkg.add("MLDatasets")
Pkg.add("ProgressMeter")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [3]:
using MLDatasets;
using BenchmarkTools;
using Random;
using ProgressMeter;

In [4]:
train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100
)

(eta = 0.01, epochs = 3, batchsize = 100)

In [5]:
train_dataset = MNIST(:train);
train_samples = size(train_dataset.features)[3];
test_dataset = MNIST(:test);
test_samples = size(test_dataset.features)[3];

In [6]:
function net(x, w1, w2, w3, w4, b1, b2, b3, b4, y)
    o1 = Conv(x, w1, b1, ReLu)
    o2 = MaxPool(o1, Constant((2,2)))
    o3 = Conv(o2, w2, b2, ReLu)
    o4 = MaxPool(o3, Constant((2,2)))
    o5 = Flatten(o4)
    o6 = Dense(w3, b3, o5, ReLu)
    o7 = Dense(w4, b4, o6, Softmax)
    E = cross_entropy_loss(o7, y)

    return topological_sort(E), o7
end

w1 = Variable(glorot_uniform((3,3,1,6)), name="w1")
b1 = Variable(zeros(6), name="b1")

w2 = Variable(glorot_uniform((3,3,6,16)), name="w2")
b2 = Variable(zeros(16), name="b2")

w3 = Variable(glorot_uniform((84, 400)), name = "w3")
b3 = Variable(zeros(84), name="b3")

w4 = Variable(glorot_uniform((10, 84)), name = "w4")
b4 = Variable(zeros(10), name="b4")

# obraz wejściowy i rzeczywista wartość
img = Variable(train_dataset[1].features, name="img")
actual_class = Variable(train_dataset[1].targets, name="actual_class")

var actual_class
 ┣━ value:    Int64
 ┗━ gradient: Nothing

In [7]:
graph, y_output = net(img, w1, w2, w3, w4, b1, b2, b3, b4, actual_class)

(Any[var actual_class
 ┣━ value:    Int64
 ┗━ gradient: Nothing, op ?(typeof(-)), var w4
 ┣━ value:    10×84 Matrix{Float64}
 ┗━ gradient: Nothing, var w3
 ┣━ value:    84×400 Matrix{Float64}
 ┗━ gradient: Nothing, var img
 ┣━ value:    28×28 Matrix{Float32}
 ┗━ gradient: Nothing, var w1
 ┣━ value:    3×3×1×6 Array{Float64, 4}
 ┗━ gradient: Nothing, var b1
 ┣━ value:    6-element Vector{Float64}
 ┗━ gradient: Nothing, op.?(typeof(Convolution)), op.?(typeof(ReLu)), const (2, 2)  …  var b3
 ┣━ value:    84-element Vector{Float64}
 ┗━ gradient: Nothing, op.?(typeof(+)), op.?(typeof(ReLu)), op.?(typeof(mul!)), var b4
 ┣━ value:    10-element Vector{Float64}
 ┗━ gradient: Nothing, op.?(typeof(+)), op.?(typeof(Softmax)), op.?(typeof(log)), op.?(typeof(*)), op.?(typeof(sum))], op.?(typeof(Softmax)))

In [8]:
w1_grad = zeros(3,3,1,6)
w2_grad = zeros(3,3,6,16)
w3_grad = zeros(84,400)
w4_grad = zeros(10, 84)

b1_grad = zeros(6)
b2_grad = zeros(16)
b3_grad = zeros(84)
b4_grad = zeros(10)



for epoch in 1:settings.epochs
    train_losses = zeros(train_samples)
    train_acc = 0
    @time @showprogress dt=1 barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',) desc="Training..." for i in 1:train_samples
    #@time for i in 1:train_samples
            img.output = reshape(train_dataset[i].features, 28,28,1,1)
            actual_class.output = onehot(train_dataset[i].targets, 0:9)
            loss_value = forward!(graph)
            train_losses[i] = loss_value
			backward!(graph)
        
            onecold(y_output.output, 0:9) == train_dataset[i].targets ? train_acc+=1 : nothing
        
            w1_grad .+= w1.gradient
            w2_grad .+= w2.gradient
            w3_grad .+= w3.gradient
            w4_grad .+= w4.gradient

            b1_grad .+= b1.gradient
            b2_grad .+= b2.gradient
            b3_grad .+= b3.gradient
            b4_grad .+= b4.gradient
            
            if i % settings.batchsize == 0
                w1.output = w1.output .- ((w1_grad/settings.batchsize)*settings.eta)
                w2.output = w2.output .- ((w2_grad/settings.batchsize)*settings.eta)
                w3.output = w3.output .- ((w3_grad/settings.batchsize)*settings.eta)
                w4.output = w4.output .- ((w4_grad/settings.batchsize)*settings.eta)

                b1.output = b1.output .- ((b1_grad/settings.batchsize)*settings.eta)
                b2.output = b2.output .- ((b2_grad/settings.batchsize)*settings.eta)
                b3.output = b3.output .- ((b3_grad/settings.batchsize)*settings.eta)
                b4.output = b4.output .- ((b4_grad/settings.batchsize)*settings.eta)

            
                w1_grad = zeros(3,3,1,6)
                w2_grad = zeros(3,3,6,16)
                w3_grad = zeros(84,400)
                w4_grad = zeros(10, 84)

                b1_grad = zeros(6)
                b2_grad = zeros(16)
                b3_grad = zeros(84)
                b4_grad = zeros(10)
            end
	end
    avg_loss = mean(train_losses)
    train_acc = train_acc/train_samples * 100
    
    @info "Train results" epoch avg_loss train_acc
end

Training... 100%|████████████████████████████████████████| Time: 0:17:196:01


1039.766392 seconds (4.82 G allocations: 5.329 TiB, 21.87% gc time, 0.53% compilation time: <1% of which was recompilation)


┌ Info: Train results
│   epoch = 1
│   avg_loss = 1.052563613590845
└   train_acc = 68.85666666666667
Training... 100%|████████████████████████████████████████| Time: 0:16:23


983.910119 seconds (4.81 G allocations: 5.329 TiB, 22.17% gc time)


┌ Info: Train results
│   epoch = 2
│   avg_loss = 0.38322017374617856
└   train_acc = 89.00833333333334
Training... 100%|████████████████████████████████████████| Time: 0:16:35


995.736291 seconds (4.81 G allocations: 5.329 TiB, 22.31% gc time)


┌ Info: Train results
│   epoch = 3
│   avg_loss = 0.285623268993559
└   train_acc = 91.68333333333332


In [9]:
test_acc = 0;
test_losses = zeros(test_samples);

@time @showprogress dt=1 barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',) desc="Testing..." for i in 1:test_samples
    img.output = reshape(test_dataset[i].features, 28,28,1,1)
    loss_value = forward!(graph)
    test_losses[i] = loss_value
    onecold(y_output.output, 0:9) == test_dataset[i].targets ? test_acc+=1 : nothing
end
test_acc = test_acc/test_samples * 100
avg_loss = mean(test_losses)

@info "Test results" avg_loss test_acc

Testing... 100%|█████████████████████████████████████████| Time: 0:00:15


 15.815641 seconds (334.15 M allocations: 44.517 GiB, 13.96% gc time)


┌ Info: Test results
│   avg_loss = 6.054891605068129
└   test_acc = 93.06
